# Homework - Module 01

The goal of this homework is to train a simple model for predicting the duration of a ride.


### Q1. Downloading the data

We'll use [the same NYC taxi dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page), but instead of "**Green** Taxi Trip Records", we'll use "**Yellow** Taxi Trip Records".

Let's download the data for January and February 2023:

In [1]:
# Download January data
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
# Download February data
!curl -O https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.4M  100 45.4M    0     0   655k      0  0:01:11  0:01:11 --:--:--  746k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 45.5M  100 45.5M    0     0   610k      0  0:01:16  0:01:16 --:--:--  545k


In [2]:
# List data files
!ls -lh yellow*

-rw-r--r--@ 1 cm-mboulou-mac  staff    45M May 19 17:57 yellow_tripdata_2023-01.parquet
-rw-r--r--@ 1 cm-mboulou-mac  staff    46M May 19 17:58 yellow_tripdata_2023-02.parquet


In [3]:
# Necessary import
import pandas as pd
import numpy as np
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

# Python version
!python --version

Python 3.9.18


Let's read the data for January and check the number of columns:

In [4]:
# Read the data
df_jan = pd.read_parquet("yellow_tripdata_2023-01.parquet")
# First rows
display(df_jan.head())

# Number of columns
print(f"The data set has {len(df_jan.columns)} columns.")

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


The data set has 19 columns.


### Q2. Computing duration

Now let's compute the `duration` variable, which should contain the duration of a ride in minutes.

In [5]:
# Feature engineering for creating the duration column
df_jan["duration"] = df_jan["tpep_dropoff_datetime"] - df_jan["tpep_pickup_datetime"]
# Convert durations in minutes
df_jan["duration"] = df_jan["duration"].apply(lambda duration_time: duration_time.total_seconds() / 60)

# The standard deviation of duration
print("The standard deviation of the trips duration in January is:", round(np.std(df_jan["duration"]), 3))

The standard deviation of the trips duration in January is: 42.594


### Q3. Dropping outliers

Next, we need to check the distribution of the `duration` variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [6]:
# Filtering for trips lasting from 1 min to 1 hour
df_jan_fil = df_jan[(df_jan["duration"] >= 1) & (df_jan["duration"] <= 60)]

# Percentage of data remaining
print(f"The fraction of the records left after dropping outliers is {round(100 * len(df_jan_fil) / len(df_jan))}%.")

The fraction of the records left after dropping outliers is 98%.


### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model. 

In [7]:
# Subset features
features = ["PULocationID", "DOLocationID"]
# Train data dictionaries - with ids as strings for one-hot encoding
train_dicts = df_jan_fil[features].astype(str).to_dict(orient = 'records')

# Initialize the dictionary vectorizer
dv = DictVectorizer()
# Fit the vectorizer getting a feature matrix
X_train = dv.fit_transform(train_dicts)

# The dimensionality of the feature matrix
print(f"Number of columns of the feature matrix: {X_train.shape[1]}.")

Number of columns of the feature matrix: 515.


### Q5. Training a model

Now let's use the feature matrix from the previous step to train a plain linear regression model with default parameters, where duration is the response variable:


In [8]:
# Target variable
target = "duration"
# Target vector
y_train = df_jan_fil[target].values

# Initialize linear regressor
lr = LinearRegression()
# Train model
lr.fit(X_train, y_train)

# Make training predictions
y_pred = lr.predict(X_train)

# RMSE on the training data
print(f"Training RMSE: {round(root_mean_squared_error(y_pred, y_train), 3)}.")

Training RMSE: 7.649.


### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023). 

In [9]:
# Read the valdation data
df_feb = pd.read_parquet("yellow_tripdata_2023-02.parquet")
# Feature engineering for creating the duration column
df_feb["duration"] = df_feb["tpep_dropoff_datetime"] - df_feb["tpep_pickup_datetime"]
# Convert durations in minutes
df_feb["duration"] = df_feb["duration"].apply(lambda duration_time: duration_time.total_seconds() / 60)
# Filtering for trips lasting from 1 min to 1 hour
df_feb_fil = df_feb[(df_feb["duration"] >= 1) & (df_feb["duration"] <= 60)]
# Validation data dictionaries
val_dicts = df_feb_fil[features].astype(str).to_dict(orient = 'records')
# Encode the feature
X_val = dv.transform(val_dicts)
# Get the target vector
y_val = df_feb_fil[target].values

# Make predictions
y_pred = lr.predict(X_val)

# RMSE on the validation data
print(f"Validation RMSE: {round(root_mean_squared_error(y_pred, y_val), 3)}.")

Validation RMSE: 7.812.


---